## Global-based Inverse Design
---

## Import Module

In [ ]:
import pandas as pd

import numpy as np
import pandas as pd
import pickle
import pymiecs

import tensorflow as tf
from tensorflow import keras



## Reload the data scaler

In [ ]:
# preprocessor path
preprocessor_path = "datasets/04_shuffle_features_scaler_MinMax(-1,1)_OneHot.pkl"
scaler_Qfwd_path = "datasets/04_shuffle_Qfwd_target_MinMaxScaler_-1-1.pkl"
scaler_Qback_path = "datasets/newData_122500/04_shuffle_Qback_target_MinMaxScaler_-1-1.pkl"


# Load the preprocessors and scalers
with open(preprocessor_path, "rb") as f:
    preprocessor = pickle.load(f)
with open(scaler_Qfwd_path, "rb") as f:
    scaler_Qfwd = pickle.load(f)
with open(scaler_Qback_path, "rb") as f:
    scaler_Qback = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/newData_122500/04_shuffle_features_scaler_MinMax(-1,1)_OneHot.pkl'

### Mie Utils

In [ ]:
# %% --- Mie
def get_Mie_spec(wavelengths, r_core, r_shell, mat_core, mat_shell, n_env):

    k0 = 2 * np.pi / wavelengths
    n_core = mat_core.get_refindex(wavelengths)
    n_shell = mat_shell.get_refindex(wavelengths)

    res = pymiecs.Q(
        k0,
        r_core=r_core,
        n_core=n_core,
        r_shell=r_shell,
        n_shell=n_shell,
        n_env=n_env.real**0.5,  # host medium must be lossless
    )
    return (
        res['qsca'],
        res['qback'],
        res['qfwd'],
    )

In [ ]:

Si = pymiecs.materials.MaterialDatabase('Si')
SiO2 = pymiecs.materials.MaterialDatabase('SiO2')
Si3N4 = pymiecs.materials.MaterialDatabase('Si3N4')
Au = pymiecs.materials.MaterialDatabase('Au')
Ag = pymiecs.materials.MaterialDatabase('Ag')
ZrO2 = pymiecs.materials.MaterialDatabase('ZrO2')
TiO2 = pymiecs.materials.MaterialDatabase('TiO2')


# Define a function to map material names to material objects
def get_material(material_name):
    if material_name == 'Si':
        return Si
    elif material_name == 'SiO2':
        return SiO2
    elif material_name == 'Au':
        return Au
    elif material_name == 'Ag':
        return Ag
    elif material_name == 'Si3N4':
        return Si3N4
    elif material_name == 'ZrO2':
        return ZrO2
    elif material_name == 'TiO2':
        return TiO2
    else:
        raise ValueError(f"Unknown material: {material_name}")

## reload Test data 

In [ ]:
hdf5_df_file = "datasets/newData_122500/02_h5_wide_122500_df_test_with_pred.h5"   # depend on what test set that forward model evalaute on
df_test = pd.read_hdf(hdf5_df_file)
df_test.head() # 2500 samples 

In [ ]:
df_test.describe()

In [ ]:
df_test['log_Qfwd'] = df_test['Q_fwd'].apply(lambda x: np.log1p(np.array(x)))
df_test['log_Qback'] = df_test['Q_back'].apply(lambda x: np.log1p(np.array(x)))

## Objective Function

In [ ]:
def calculate_mie_scattering_error(
    r_core,
    r_shell,
    core_material,
    shell_material,
    target_Qfwd_transformed,
    target_Qback_transformed,
):
    # Define wavelengths
    wavelengths = np.linspace(400, 800, 64)

    # Get material properties using your material function
    mat_core = get_material(core_material)
    mat_shell = get_material(shell_material)

    # Calculate Mie scattering using the new package
    Qsca, Qback, Qfwd = get_Mie_spec(
        wavelengths=wavelengths,
        r_core=r_core,
        r_shell=r_shell,
        mat_core=mat_core,
        mat_shell=mat_shell,
        n_env=1.0,  # Medium refractive index
    )

    # Apply log1p transformation to Mie Qfwd and Qback
    log_mie_Qfwd = np.log1p(Qfwd)
    log_mie_Qback = np.log1p(Qback)

    # Now, apply MinMax scaling to log-transformed Mie values
    log_mie_Qfwd_transformed = scaler_Qfwd.transform(
        log_mie_Qfwd.reshape(1, -1)
    )  # Reshape to (1, 64)
    log_mie_Qback_transformed = scaler_Qback.transform(log_mie_Qback.reshape(1, -1))

    # Compute the MSE between the predicted and target values
    mse_Qfwd = np.mean((log_mie_Qfwd_transformed - target_Qfwd_transformed) ** 2)
    mse_Qback = np.mean((log_mie_Qback_transformed - target_Qback_transformed) ** 2)

    # Total error is the sum of MSEs
    total_error = mse_Qfwd + mse_Qback
    return total_error

In [ ]:
# Define the objective function (your Mie calculations and comparison with target)
def objective_function(
    geometry_params, target_Qfwd_transformed, target_Qback_transformed
):
    # Your Mie calculations and error computation go here
    r_core_real, r_shell_real, mat_core, mat_shell = geometry_params

    # (Example calculation) Total error is the sum of MSEs for Qfwd and Qback
    total_error = calculate_mie_scattering_error(
        r_core_real,
        r_shell_real,
        mat_core,
        mat_shell,
        target_Qfwd_transformed,
        target_Qback_transformed,
    )

    return total_error  # The optimizer will minimize this error

## Run global Optimization

In [ ]:
import nevergrad as ng
from concurrent import futures
import time

In [ ]:
# Define the search space for core radius, shell thickness, core material, shell material
parametrization = ng.p.Tuple(
    ng.p.Scalar(lower=1, upper=100),  # Real core radius (nm)
    ng.p.Scalar(lower=6, upper=200),   # Real shell thickness (nm)
    ng.p.Choice(["Si", "SiO2", "Au", "Ag", "Si3N4","ZrO2", "TiO2"]),  # Core material
    ng.p.Choice(["Si", "SiO2", "Au", "Ag", "Si3N4","ZrO2", "TiO2"])   # Shell material
)

In [ ]:
# Initialize lists to store results and runtimes
sample_runtimes = []
results = []
# Define the number of workers for parallel processing and budget
num_workers = 25
budget = 1000  # Set the number of function evaluations (budget)

# Loop over each sample in the subset of synthetic data
for idx, row in df_test.iterrows():
    print(f"Processing sample {idx + 1}/{len(df_test)}")

    # Extract the target data for this sample
    target_Qfwd = np.array(row["log_Qfwd"]).reshape(1, -1)
    target_Qback = np.array(row["log_Qback"]).reshape(1, -1)

    # Apply MinMax scaling to the target
    target_Qfwd_transformed = scaler_Qfwd.transform(target_Qfwd)
    target_Qback_transformed = scaler_Qback.transform(target_Qback)
    obj_fun = lambda params: objective_function(
        params, target_Qfwd_transformed, target_Qback_transformed
    )

    # Initialize Differential Evolution optimizer with specified crossover and population size
    optim_algo = ng.families.DifferentialEvolution(
        crossover="twopoints", popsize=num_workers
    )

    # Run the optimization process for this sample
    optimizer = optim_algo(parametrization, budget=budget)

    # start time
    start_time = time.time()

    # Use a ThreadPoolExecutor for parallel batch processing
    with futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
        recommendation = optimizer.minimize(obj_fun, executor=executor)

    # end time
    end_time = time.time()
    sample_runtime = end_time - start_time

    # Append runtime and best geometry to the results
    sample_runtimes.append({"Sample": idx + 1, "Runtime (seconds)": sample_runtime})
    print(f"Runtime for sample {idx}: {sample_runtime:.2f} seconds")

    best_geometry = recommendation.value
    best_loss = recommendation.loss
    print(f"Best geometry for sample {idx}: {best_geometry}")
    print(f"Best loss for sample {idx}: {best_loss}")
    print("-------------------------------------------------------")

    # Append results to the list
    result = {
        "Sample Index": idx + 1,
        "Best Geometry": best_geometry,
        "Loss": best_loss,
        "Runtime (seconds)": sample_runtime,
    }

    results.append(result)

"""
##########################################################################################
####################### Convert the runtimes into a DataFrame ############################
##########################################################################################
"""
# Convert the results and runtimes to DataFrames and save
results_df = pd.DataFrame(results)
runtime_df = pd.DataFrame(sample_runtimes)

# Optionally, save the DataFrames to files
sample_runtime_path = "runtime/global/01_testSetNew_122500_runtime_2500.pkl"
result_df_path = "best_geometries/global/02_best_geo_testsetNew_122500_df_2500.pkl"

with open(sample_runtime_path, "wb") as f:
    pickle.dump(runtime_df, f)

with open(result_df_path, "wb") as f:
    pickle.dump(results_df, f)

In [ ]:
'''
##########################################################################################
####################### save the best geo DataFrame ######################################
##########################################################################################
'''

with open('best_geometries/global/02_best_geo_testsetNew_122500_df_2500.pkl', 'rb') as f:
    best_df = pickle.load(f)
    
best_df.head()

In [ ]:
# Split the 'Best Geometry' column into multiple columns
best_df[["r_core", "r_shell", "mat_core", "mat_shell"]] = pd.DataFrame(
    best_df["Best Geometry"].tolist(), index=best_df.index
)

# Drop the 'Best Geometry' column if you no longer need it
best_df = best_df.drop(columns=["Best Geometry"])
best_df = best_df[
    [
        "mat_core",
        "mat_shell",
        "r_core",
        "r_shell",
        "Sample Index",
        "Runtime (seconds)",
        "Loss",
    ]
]

# Display the updated DataFrame
best_df

In [ ]:

"""
###########################################################################################
Now use the optimized geometries to calculate Mie Qfwd and Qback using  Mie theory function
###########################################################################################
"""

# wavelengths = np.linspace(400, 800, 128)  # From 400 nm to 800 nm
wavelengths = np.linspace(400, 800, 64)  # From 400 nm to 800 nm
n_env = 1.0

mie_Qfwd_list, mie_Qback_list = [], []
# Loop through each row in the optimized geometries DataFrame
for idx, row in best_df.iterrows():
    mat_core = get_material(row["mat_core"])
    mat_shell = get_material(row["mat_shell"])
    r_core = row["r_core"]
    r_shell = row["r_shell"]

    # Calculate the Mie spectrum for this geometry
    _, Qback, Qfwd = get_Mie_spec(
        wavelengths, r_core, r_shell, mat_core, mat_shell, n_env
    )

    mie_Qfwd_list.append(Qfwd)
    mie_Qback_list.append(Qback)

# Convert the Mie Qfwd and Qback to DataFrame or arrays if needed
mie_Qfwd_array = np.array(mie_Qfwd_list)
mie_Qback_array = np.array(mie_Qback_list)

# Validation: Check dimensions
assert len(mie_Qfwd_array) == len(best_df), "Mismatch in dimensions for Qfwd!"
assert len(mie_Qback_array) == len(best_df), "Mismatch in dimensions for Qback!"

# Add the Mie results to the DataFrame
best_df["mie_Qfwd"] = mie_Qfwd_array.tolist()
best_df["mie_Qback"] = mie_Qback_array.tolist()

best_df

### Also compare Mie to foward model predictions

## Reload the forward model

### Define the own resblock class

keras requires custom classes to be defined for being able to reload

In [ ]:
@keras.utils.register_keras_serializable()
class ResBlock1D(keras.Model):
    def __init__(self, filters, kernel_size=3, convblock=False, **kwargs):
        super(ResBlock1D, self).__init__(**kwargs)

        # setup all necessary layers
        self.conv1 = keras.layers.Conv1D(filters, kernel_size, padding="same")
        self.bn1 = keras.layers.BatchNormalization()

        self.conv2 = keras.layers.Conv1D(filters, kernel_size, padding="same")
        self.bn2 = keras.layers.BatchNormalization()

        self.relu = keras.layers.LeakyReLU(negative_slope=0.1)

        self.convblock = convblock
        if self.convblock:
            self.conv_shortcut = keras.layers.Conv1D(filters, 1)

    def call(self, input_tensor, training=False):
        x = self.conv1(input_tensor)
        x = self.bn1(x, training=training)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn2(x, training=training)

        # add shortcut. optionally pass it through a Conv
        if self.convblock:
            x_sc = self.conv_shortcut(input_tensor)
        else:
            x_sc = input_tensor
        x += x_sc
        return self.relu(x)

    def get_config(self):
        base_config = super().get_config()
        return {
            "filters": self.filters,
            "kernel_size": self.kernel_size,
            "convblock": self.convblock,
            **base_config,
        }

In [ ]:
forward_path = "models/resnet/newData_122500/04_keras3_increaseBatch_decayLR_hybridMode.keras"
forward_model = keras.models.load_model(forward_path)

NameError: name 'keras' is not defined

In [ ]:
"""
#########################################################################################
############### Use forward model to predict on the optimized geometries  ###############
#########################################################################################
"""

# Ensure that the columns are ordered the same way as the input during training
categorical_features = [
    "mat_core",
    "mat_shell",
]  # Update these names based on your data
numerical_features = ["r_core", "r_shell"]

# Use the preprocessor to transform the data
X_optimized_preprocessed = preprocessor.transform(
    best_df[categorical_features + numerical_features]
)

# Predict using the forward model
y_pred = forward_model.predict(X_optimized_preprocessed)

# Separate predictions for Qfwd and Qback
y_pred_Qfwd = y_pred[..., 0]  # First output for Qfwd predictions
y_pred_Qback = y_pred[..., 1]  # Second output for Qback predictions

# Inverse transform the predicted Qfwd and Qback to get the original scale
predicted_Qfwd_original = scaler_Qfwd.inverse_transform(y_pred_Qfwd)
predicted_Qback_original = scaler_Qback.inverse_transform(y_pred_Qback)

# Apply expm1 to reverse the log1p transformation
predicted_Qfwd_original = np.expm1(predicted_Qfwd_original)  # Reverse log1p for Qfwd
predicted_Qback_original = np.expm1(predicted_Qback_original)  # Reverse log1p for Qback

# Display the predicted Qfwd and Qback in original scale
print("Predicted Qfwd (Original Scale):", predicted_Qfwd_original)
print("Predicted Qback (Original Scale):", predicted_Qback_original)

# Add the predicted values (in original scale) to the DataFrame
best_df["predicted_Qfwd"] = predicted_Qfwd_original.tolist()
best_df["predicted_Qback"] = predicted_Qback_original.tolist()


In [ ]:
best_df['log_Qfwd'] = best_df['mie_Qfwd'].apply(lambda x: np.log1p(np.array(x)))
best_df['log_Qback'] = best_df['mie_Qback'].apply(lambda x: np.log1p(np.array(x)))
best_df.head()

In [ ]:
"""
#########################################################################################
############### Save the Best Geometries, predicted, Mie into Dataframe   ###############
#########################################################################################
"""

# save final_best_geometries_df to pkl file
best_geometries_path = (
    "best_geometries/global/02_best_geo_testsetNew_122500_df_predicted_mie_2500.pkl"
)

# Saving DataFrames
with open(best_geometries_path, "wb") as f:
    pickle.dump(best_df, f)